# UHFF Visualization - Android APK Builder

This notebook builds your UHFF (Universal Harmonic Field Framework) visualization app into an Android APK using Google Colab.

## Features
- ✅ Touch controls for Android devices
- ✅ Virtual on-screen buttons
- ✅ Responsive UI design
- ✅ Full pygame-based visualization

## Build Time
- First build: ~20-30 minutes (downloads Android NDK)
- Subsequent builds: ~5-10 minutes

---

In [ ]:
# 🚀 STEP 1: Setup Environment
print("Setting up Android build environment...")

# Update system and install dependencies
!apt-get update -qq
!apt-get install -y -qq openjdk-17-jdk wget unzip git

# Install Python dependencies
!pip install buildozer cython

print("✅ Environment setup complete!")

In [ ]:
# 📦 STEP 2: Setup Android SDK
import os
import zipfile
from pathlib import Path

# Create Android SDK directory
android_home = Path.home() / "android-sdk"
android_home.mkdir(exist_ok=True)

# Download and setup Android command line tools
if not (android_home / "cmdline-tools" / "latest").exists():
    print("📥 Downloading Android SDK...")
    
    # Download command line tools
    !wget -q https://dl.google.com/android/repository/commandlinetools-linux-10406996_latest.zip -O cmdtools.zip
    
    # Extract and organize
    with zipfile.ZipFile('cmdtools.zip', 'r') as zip_ref:
        zip_ref.extractall(android_home / "cmdline-tools")
    
    # Move to correct structure
    import shutil
    shutil.move(
        str(android_home / "cmdline-tools" / "cmdline-tools"),
        str(android_home / "cmdline-tools" / "latest")
    )
    os.remove('cmdtools.zip')
    print("✅ Android SDK command line tools installed")

# Set environment variables
os.environ['ANDROID_HOME'] = str(android_home)
os.environ['PATH'] = f"{android_home}/cmdline-tools/latest/bin:{android_home}/platform-tools:{os.environ['PATH']}"

# Install SDK components
if not (android_home / "platform-tools").exists():
    print("📦 Installing Android SDK components...")
    !yes | {android_home}/cmdline-tools/latest/bin/sdkmanager --licenses
    !{android_home}/cmdline-tools/latest/bin/sdkmanager "platform-tools" "platforms;android-33" "build-tools;33.0.2"
    print("✅ Android SDK ready!")
else:
    print("✅ Android SDK already installed")

In [ ]:
# 📁 STEP 3A: Clone Your Repository (Option 1)
# Uncomment and modify the line below to clone your GitHub repository:

# !git clone https://github.com/3rdeyesamurai/ether.git uhff-project

print("If you have a GitHub repository, uncomment and run the git clone command above.")
print("Otherwise, use the file upload method in the next cell.")

In [ ]:
# 📁 STEP 3B: Upload Files Manually (Option 2)
from google.colab import files
import zipfile
import shutil

# Create project directory
project_dir = Path("uhff-project")
project_dir.mkdir(exist_ok=True)

print("Choose upload method:")
print("1. Upload individual files")
print("2. Upload ZIP file of entire project")

choice = input("Enter choice (1 or 2): ")

if choice == "1":
    print("Please upload your Python files:")
    uploaded = files.upload()
    
    # Move files to project directory
    for filename in uploaded.keys():
        shutil.move(filename, project_dir / filename)
        print(f"✅ Moved {filename}")

elif choice == "2":
    print("Please upload your project as a ZIP file:")
    uploaded = files.upload()
    
    # Extract ZIP files
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(project_dir)
            os.remove(filename)
            print(f"✅ Extracted {filename}")

# Show uploaded files
print("\n📋 Project files:")
for item in project_dir.rglob("*"):
    if item.is_file():
        print(f"  - {item.relative_to(project_dir)}")

In [ ]:
# 🔧 STEP 4: Create/Update buildozer.spec for Android
os.chdir(project_dir)

# Create optimized buildozer.spec if it doesn't exist
buildozer_content = """[app]
title = UHFF Visualization App
package.name = uhffvis
package.domain = com.uhff
source.dir = .
source.include_exts = py,png,jpg,kv,atlas,json
version = 1.0
requirements = python3,pygame,numpy
orientation = landscape,portrait
fullscreen = 1

[buildozer]
log_level = 2

# Android specific
android.api = 33
android.minapi = 21
android.ndk = 25b
android.ndk_api = 21
android.archs = arm64-v8a, armeabi-v7a
android.accept_sdk_license = True
p4a.bootstrap = sdl2
"""

if not Path("buildozer.spec").exists():
    with open("buildozer.spec", "w") as f:
        f.write(buildozer_content)
    print("✅ Created buildozer.spec")
else:
    print("✅ Using existing buildozer.spec")

# Show project structure
print("\n📁 Final project structure:")
!ls -la

In [ ]:
# 🏗️ STEP 5: Build APK
print("🚀 Starting APK build...")
print("⏱️ This will take 15-30 minutes for the first build!")
print("☕ Perfect time for a coffee break!")

# Build the APK
!buildozer android debug

print("\n🎉 Build completed!")

In [ ]:
# 📱 STEP 6: Check Build Results
from pathlib import Path

bin_dir = Path("bin")
if bin_dir.exists():
    apk_files = list(bin_dir.glob("*.apk"))
    if apk_files:
        print("🎉 APK files generated:")
        for apk in apk_files:
            size_mb = apk.stat().st_size / 1024 / 1024
            print(f"  📱 {apk.name} ({size_mb:.1f} MB)")
        
        print("\n✅ Build successful!")
        print("📥 You can download the APK in the next cell.")
    else:
        print("❌ No APK files found. Check build logs above for errors.")
else:
    print("❌ Build directory not found. Build may have failed.")

# Show build logs if there were errors
if Path(".buildozer/android/platform/build-arm64-v8a/build.log").exists():
    print("\n📋 Recent build log (last 20 lines):")
    !tail -20 .buildozer/android/platform/build-arm64-v8a/build.log

In [ ]:
# 📥 STEP 7: Download APK
from google.colab import files

bin_dir = Path("bin")
if bin_dir.exists():
    apk_files = list(bin_dir.glob("*.apk"))
    
    if apk_files:
        print("📥 Downloading APK files...")
        for apk_file in apk_files:
            print(f"⬇️ Downloading {apk_file.name}...")
            files.download(str(apk_file))
        
        print("\n✅ Downloads complete!")
        print("\n📱 Installation instructions:")
        print("1. Transfer APK to your Android device")
        print("2. Enable 'Install from Unknown Sources' in Android settings")
        print("3. Tap the APK file to install")
        print("4. Enjoy your UHFF visualization app!")
    else:
        print("❌ No APK files to download")
else:
    print("❌ No build output found")

## 🛠️ Utility Functions

Use these cells for troubleshooting and maintenance:

In [ ]:
# 🧹 Clean build cache (run if build fails)
!buildozer android clean
print("✅ Build cache cleaned")

In [ ]:
# 🔍 Debug: Check project files
print("📁 Project files:")
!ls -la

print("\n🐍 Python files:")
!find . -name "*.py" -type f

In [ ]:
# 🔍 Debug: Check build requirements
print("🔧 Environment check:")
print(f"Java: {!java -version}")
print(f"Python: {!python3 --version}")
print(f"Buildozer: {!buildozer version}")
print(f"Android Home: {os.environ.get('ANDROID_HOME', 'Not set')}")